In [ ]:
 
# mpiexec -np 4 python main.py -country ./language.json -data ./tinyTwitter.json
import couchdb
from couchdb.design import ViewDefinition

def getConnection():

    # Connect to the CouchDB instance running on localhost with username and password
    username = 'admin'
    password = r'1dTY1!PWM2'
    ip = '45.113.234.176'
    url = 'http://{}:{}@{}:5984/'.format(username, password, ip)
    return couchdb.Server(url)
 

In [79]:

from time import sleep


def delete_view(db_name,design_doc_name,view_name):
    try:
        couch=getConnection()
        db=couch[db_name]
        doc_id = '_design/' + design_doc_name
        doc = db.get(doc_id)
        del doc['views'][view_name]
        db.save(doc)
        sleep(0.1)
        
    except Exception as e:
        pass
    print('deleted','view:',design_doc_name+'/'+view_name,'in db:',db_name,)
        
def create_view(db_name, design_doc,view_name, map_func,reduce_fun=None):
    couch = getConnection()
    db=couch[db_name]
    options = {}
   # Create a new view definition object
    delete_view(db_name,design_doc,view_name)
    sleep(2)
    if reduce_fun!=None:
        options = {'reduce':reduce_fun}
    view_def = ViewDefinition(design_doc, view_name, map_func,options=options,reduce_fun=reduce_fun)
    view_def.sync(db)
    sleep(0.1)
    print('Created or updated','view: ',design_doc+'/'+view_name,'in db: ',db_name,)

def create_bulk_view(db_names, design_doc_name, view_name, map_fun,reduce_fun=None):
    for db_name in db_names:
        couch = getConnection()
        db = couch[db_name]
        create_view(db_name, design_doc_name, view_name, map_fun,reduce_fun=reduce_fun)
         
def delete_bulk_view(db_names, design_doc_name, view_name):
    for db_name in db_names:
        couch = getConnection()
        db = couch[db_name]
        delete_view(db_name, design_doc_name, view_name)
          

 
db_names = [  'huge-twitter-2022-2', 'huge-twitter-2022-3',
            'huge-twitter-2022-4', 'huge-twitter-2022-5', 'huge-twitter-2022-6',
            'huge-twitter-2022-7', 'huge-twitter-2022-8',  ]

In [ ]:
# a view to  tweets about war
couch = getConnection()


view_name = 'war_related_tweets2'
map_function = '''function (doc) {
    if (doc.RUwar === true)
                    emit(doc._id, doc);
                    }'''
db = couch['mastodon_data']
create_view('mastodon_data',  view_name, map_function,)
 
# query the view with a specified key
view_results = db.view(view_name+'/'+view_name )
for view_result in view_results:
    print(view_result.value['content'])
 

In [ ]:
# a view to  tweets about language
couch = getConnection()
design_doc_name='war_related_tweets'
view_name = 'count_language'
map_fun = '''function(doc) {
    if (doc.language) {
        emit(doc.language, 1);
    }
}'''

reduce_fun = '''function(keys, values, rereduce) {
        return sum(values);
    }'''
db = couch['mastodon_data']
create_view('mastodon_data', design_doc_name, view_name, map_fun,reduce_fun=reduce_fun)
 


In [ ]:
# query the view with a specified key
couch=getConnection()
db=couch['mastodon_data']
view_results = db.view(design_doc_name+'/'+view_name ,group=True)
for view_result in view_results:
    print(view_result.key,view_result.value) 
# delete_view('mastodon_data',design_doc_name,view_name)

In [ ]:
 
# a view to  tweets about language
couch = getConnection()
design_doc_name='war_related_tweets'
view_name = 'tweets_about_war'
db_name='huge-twitter'
map_fun = '''function (doc) {
    let tokens = doc.tokens.split('|');
    tokens = tokens.map((token) => token.toLowerCase());
    if ("russia" in tokens || "war" in tokens || "ukraine" in tokens || "nato" in tokens || "putin" in tokens) {
        if (doc.lang === "en") {
            emit(null, doc);
        }
    }
}'''
 
reduce_fun=None

create_view(db_name, design_doc_name, view_name, map_fun, )
 


In [76]:
 
# a views to  tweets about war
 
design_doc_name='war_related_tweets'
view_name = 'tweets_about_war'
 
map_fun = '''function (doc) {
    let tokens = doc.tokens.split('|');
    tokens = tokens.map((token) => token.toLowerCase());
    if ("russia" in tokens || "war" in tokens || "ukraine" in tokens || "nato" in tokens || "putin" in tokens) {
        if (doc.lang === "en") {
            emit(null, doc);
        }
    }
}'''
reduce_fun=None
# create_bulk_view(db_names, design_doc_name, view_name, map_fun, )
# delete_bulk_view(db_names,design_doc_name,view_name)
create_view('huge-twitter-2022-2', design_doc_name, view_name, map_fun, )

deleted db:  huge-twitter-2022-2 view:  war_related_tweets/tweets_about_war
Created or updated db:  huge-twitter-2022-2 view:  war_related_tweets/tweets_about_war


In [90]:
 
# a views to  tweets about war
 
design_doc_name='war_related_tweets'
view_name = 'tweets_about_war'
 
map_fun = '''

function (doc) {
    let tokens = doc.tokens.split('|');
    tokens = tokens.map((token) => token.toLowerCase());
    const relatedWords = ["russia", "ukraine", "nato", "putin", "ukr", "nazi"];
    const isRelated = tokens.some((token) => relatedWords.includes(token));
    if (tokens.length>0 && isRelated) {
        
        emit(null, doc)
    }
}
  
  '''
reduce_fun=None
# create_bulk_view(db_names, design_doc_name, view_name, map_fun, )
# delete_bulk_view(db_names,design_doc_name,view_name)
create_view('huge-twitter', design_doc_name, view_name, map_fun, )

deleted view: war_related_tweets/tweets_about_war in db: huge-twitter
Created or updated view:  war_related_tweets/tweets_about_war in db:  huge-twitter


In [85]:
# query the view with a specified key
couch=getConnection()
db=couch[db_name]
view_results = db.view(design_doc_name+'/'+view_name ,)
for view_result in view_results:
    print(view_result.key,view_result.value['tokens']) 
# delete_view('mastodon_data',design_doc_name,view_name)

None putin|please|leave|well|alone|the|Ukraine
None Big|talk|coming|from|the|lady|that|prosecuting|war|heroes|for|whistleblowing|corruption|enacted|her|friends|amp|family
None proving|his|imperialist|credentials|today|beating|Stop|the|War|Coalition|heads|genuflect|talk|about|flogging|dead|horse|Next|time|have|anti|war|movement|inUK|prioritise|supporting|those|who|move|from|protest|toNonviolentResistance
None Nah|think|inUK|led|the|anti|war|movement|into|dead|end|cattle|drive|marches|through|empty|streets|They|failed|support|military|amp|civilians|before|the|courts|for|resisting|the|War|Failed|block|hwood|armer|stooge
None People|power|political|orNGO|milk|amp|manipulate|anti|war|phenomenon|when|arises|haven|had|movement|since|the|1980|armer|over|friend|year|detention|shifty|fuck|presently|marching|lock|step|war
None Seems|some|confusion|usually|the|defendant|prisoner|amp|beaten|when|there|mobilisation|for|war|know|fit|when|see|one|amp|has|paid|the|heaviest|price|theUK|for|opposition|St

In [97]:
# word frequency

 
# a views to  tweets about war
 
design_doc_name='war_related_tweets'
view_name = 'token_frequency'
 
map_fun = '''

function  (doc) {
    let tokens = doc.tokens.split('|');
    tokens = tokens.map((token) => token.toLowerCase());
    const relatedWords = ["russia", "ukraine", "nato", "putin", "ukr",
        "nazi", "crimea", "russo-ukrainian", "bakhmut", "wagner", "belgorod",
        "zhytomyr", "kherson", "dnipro", "nato", "donetsk", "zelensky"];
    const isRelated = tokens.some((token) => relatedWords.includes(token));
    if (isRelated) {
        for (let token of tokens) {
            emit(token, 1);
        }
    }

}
  
  '''
reduce_fun='_count'
# create_bulk_view(db_names, design_doc_name, view_name, map_fun, )
# delete_bulk_view(db_names,design_doc_name,view_name)
create_view('huge-twitter-2022-8', design_doc_name, view_name, map_fun, reduce_fun=reduce_fun)

deleted view: war_related_tweets/token_frequency in db: huge-twitter-2022-8
Created or updated view:  war_related_tweets/token_frequency in db:  huge-twitter-2022-8


In [98]:
# query the view with a specified key
import nltk
from nltk.corpus import stopwords

# Download the set of stop words
nltk.download('stopwords')

# Get the set of English stop words
stop_words = set(stopwords.words('english'))
couch=getConnection()
db=couch[db_name]

view_results = db.view(design_doc_name+'/'+view_name ,group=True)
filtered_view_results = [view_result for view_result in view_results if view_result.key not in stop_words]
sorted_view_results = sorted(filtered_view_results, key=lambda x: x.value, reverse=True)

for view_result in sorted_view_results:
    print(view_result.key, view_result.value) 
# delete_view('mastodon_data',design_doc_name,view_name)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhihao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ResourceNotFound: ('not_found', 'missing_named_view')